In [6]:
# importing packages
import pandas as pd
import re

from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()

from selenium.webdriver.support.wait import WebDriverWait

from warnings import warn

In [7]:
# replace variables here.
url = "https://www.linkedin.com/jobs/search/?geoId=100565514&keywords=data%20science&location=Belgium"
no_of_jobs = 50

In [8]:
driver = webdriver.Chrome(r'C:\Users\Santosh\Linkedin Jobs Insgiht\Indeed_LinkedIn_Scraper-main\AutomateJobSearch\chromedriver.exe')
driver.get(url)
wait = WebDriverWait(driver, 10)
action = ActionChains(driver)

In [9]:
# this will open up new window with the url provided above 
driver = webdriver.Chrome(r'C:\Users\Santosh\Linkedin Jobs Insgiht\Indeed_LinkedIn_Scraper-main\AutomateJobSearch\chromedriver.exe')
driver.get(url)
sleep(3)
action = ActionChains(driver)

In [10]:
# to show more jobs. Depends on number of jobs selected
i = 1
while i <= (no_of_jobs/50): 
    driver.find_element_by_xpath('/html/body/main/div/section/button').click()
    i = i + 1
    sleep(5)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=87.0.4280.141)


In [11]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

You are scraping information about 25 jobs.


In [12]:
# setting up list for job information
job_id = []
post_title = []
company_name = []
post_date = []
job_location = []
job_desc = []
level = []
emp_type = []
functions = []
industries = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # job title
    job_titles = job.find("span", class_="screen-reader-text").text
    post_title.append(job_titles)
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)
    
    # company name
    company_names = job.select_one('img')['alt']
    company_name.append(company_names)
    
    # job location
    job_locations = job.find("span", class_="job-result-card__location").text
    job_location.append(job_locations)
    
    # posting date
    post_dates = job.select_one('time')['datetime']
    post_date.append(post_dates)

    
# for loop for job description and criterias
for x in range(1,len(job_id)+1):
    
    # clicking on different job containers to view information about the job
    job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
    driver.find_element_by_xpath(job_xpath).click()
    sleep(3)
    
    # job description
    jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div'
    job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
    job_desc.append(job_descs)
    
    # job criteria container below the description
    job_criteria_container = lxml_soup.find('ul', class_ = 'job-criteria__list')
    all_job_criterias = job_criteria_container.find_all("span", class_='job-criteria__text job-criteria__text--criteria')
    
    # Seniority level
    seniority_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[1]'
    seniority = driver.find_element_by_xpath(seniority_xpath).text.splitlines(0)[1]
    level.append(seniority)
    
    # Employment type
    type_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[2]'
    employment_type = driver.find_element_by_xpath(type_xpath).text.splitlines(0)[1]
    emp_type.append(employment_type)
    
    # Job function
    function_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[3]'
    job_function = driver.find_element_by_xpath(function_xpath).text.splitlines(0)[1]
    functions.append(job_function)
    
    # Industries
    industry_xpath = '/html/body/main/section/div[2]/section[2]/ul/li[4]'
    industry_type = driver.find_element_by_xpath(industry_xpath).text.splitlines(0)[1]
    industries.append(industry_type)
    
    x = x+1    

In [21]:
# to check if we have all information
print(len(job_id))
print(len(post_date))
print(len(company_name))
print(len(post_title))
print(len(job_location))
print(len(job_desc))
print(len(level))
print(len(emp_type))
print(len(functions))
print(len(industries))

24
24
24
24
24
24
24
24
24
24


In [22]:
# creating a dataframe
job_data = pd.DataFrame({'Job ID': job_id,
'Date': post_date,
'Company Name': company_name,
'Post': post_title,
'Location': job_location,
'Description': job_desc,
'Level': level,
'Type': emp_type,
'Function': functions,
'Industry': industries
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())
job_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Job ID        24 non-null     object
 1   Date          24 non-null     object
 2   Company Name  24 non-null     object
 3   Post          24 non-null     object
 4   Location      24 non-null     object
 5   Description   24 non-null     object
 6   Level         24 non-null     object
 7   Type          24 non-null     object
 8   Function      24 non-null     object
 9   Industry      24 non-null     object
dtypes: object(10)
memory usage: 2.0+ KB
None


,Job ID,Date,Company Name,Post,Location,Description,Level,Type,Function,Industry
0,2305809129,2020-10-18,Cognizant,Junior Data Scientist,"Antwerp, Flemish Region, Belgium",Our Company Cognizant is one of the world's l...,Entry level,Full-time,EngineeringInformation Technology,Information Technology and ServicesStaffing an...
1,2355617771,2021-01-08,KBC Bank &amp; Verzekering,Data Scientist,"Flemish Region, Belgium",What does this job entail? In a changing mark...,Entry level,Full-time,AnalystInformation TechnologyEngineering,Information ServicesResearch
2,2329074638,2020-12-15,EY,Junior Data Analyst,"Brussels, Brussels Region, Belgium","EY is a global leader in assurance, tax, trans...",Entry level,Full-time,Analyst,Information ServicesInformation Technology and...
3,2330688025,2020-12-17,Brussels Airport Company,Data Scientist,"Brussels, Brussels Region, Belgium",Apply Here Back Bouw jij mee aan de luchthave...,Entry level,Full-time,EngineeringInformation Technology,ConstructionInformation Technology and Service...
4,2302899305,2020-10-17,KPMG Deutschland,(Junior) Data Scientist / Data Analyst (w/m/d),"Kiel, Flemish Region, Belgium",Als (Junior) Data Scientist / Data Analyst (w/...,Entry level,Full-time,EngineeringInformation Technology,DesignInternetManagement Consulting


In [23]:
job_data.to_csv('LinkedIn Job Data.csv', index=0)